# gwosc模块

`gwosc`是一个用于查询引力波数据信息的python包

## gwosc.datasets
`gwosc.datasets` 模块提供搜寻数据集，gps时间返回等功能

### gwosc.datasets.find_datasets 

`gwosc.datasets.find_datasets` 提供寻找LIGO-virgo数据集的功能，还可以利用GPS时间、正则表达式进行筛选。可以进行引力波事件查询

**说明**

```python
find_datasets(detector=None, type=None, segment=None, match=None, host='https://www.gw-openscience.org')
    Find datasets available on the given GW open science host
    
Parameters
----------
detector : `str`, optional
    prefix of GW detector

type : `str`, optional
    type of datasets to restrict, one of ``'run'``, ``'event'``, or
    ``'catalog'``

segment : 2-`tuple` of `int`, `None`, optional
    a GPS ``[start, stop)`` interval to restrict matches to;
    datasets will match if they overlap at any point

match : `str`, optional
    regular expression string against which to match datasets

host : `str`, optional
    the URL of the LOSC host to query, defaults to losc.ligo.org

Returns
-------
datasets : `list` of `str`
    the names of all matched datasets, possibly empty

```

#### 例子

例如，我们可以寻找哪些数据集是可用的。在O2运行过程中，一共有25个事件数据集可用。11个是确定事件(前缀为gw), 其他的是边缘事件，可以参考[GWTC-1](https://www.gw-openscience.org/GWTC-1/). 

In [39]:
from gwosc.datasets import find_datasets #导入模块
gwsets = find_datasets()
print(gwsets)

['151008', '151012A', '151116', '161202', '161217', '170208', '170219', '170405', '170412', '170423', '170616', '170630', '170705', '170720', 'GW150914', 'GW151012', 'GW151226', 'GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823', 'GWTC-1-confident', 'GWTC-1-marginal', 'O1', 'O1_16KHZ', 'O2_16KHZ_R1', 'O2_4KHZ_R1', 'S5', 'S6']


还可以设置detector参数来选定指定探测器探测的的事件

In [33]:
print(find_datasets(type="event", detector="V1")) #virgo
print(find_datasets(type="event", detector="G1")) #GEO600

['GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818']
['GW170817']


`find_datasets` 也可以接受 `segment` 关键字来限制结果的GPS time:

In [35]:
print(find_datasets(type="event", detector="L1", segment=(1164556817, 1187733618)))

['161202', '161217', '170208', '170219', '170405', '170412', '170423', '170616', '170630', '170705', '170720', 'GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']


### gwosc.datasets.event_gps

`gwosc.datasets.event_gps` 对于开放数据中的给定引力波事件返回其GPS时间

**说明**

```python
event_gps(event, host='https://www.gw-openscience.org')
    Returns the GPS time of an open-data event
    
    Parameters
    ----------
    event : `str`
        the name of the event to query
    
    host : `str`, optional
        the URL of the LOSC host to query, defaults to losc.ligo.org
    
    Returns
    -------
    gps : `float`
        the GPS time of this event
     
```

#### 例子

In [67]:
from gwosc.datasets import event_gps
gps = event_gps('GW170817')
print(gps)

1187008882.4


<div class="alert alert-info">返回的时间都是GPS时间，是从世界时1980年1月6日午夜(00:00)开始的秒数. GWOSC 提供了一个 <a href="https://www.gw-openscience.org/gps/">GPS 时间转换器</a> 你可以进行GPS时间和世界时的转换，或者你也可以用 <a href="https://gwpy.github.io/docs/stable/time/"><code>gwpy.time</code></a>.</div>

### gwosc.datasets.run_segment

我们也可以用`gwosc.datasets.run_segment` 查询探测器某次运行的GPS时间段

**说明**
```python
run_segment(run, host='https://www.gw-openscience.org')
    Returns the GPS ``[start, stop)`` interval covered by a run dataset
    
    Parameters
    ----------
    event : `str`
        the name of the event to query
    
    host : `str`, optional
        the URL of the LOSC host to query, defaults to losc.ligo.org
    
    Returns
    -------
    start, end : `int`
        the GPS ``[start, end)`` interval covered by this run dataset
```                  

#### 例子

In [65]:
from gwosc.datasets import run_segment
print(run_segment('O1'))

(1126051217, 1137254417)


## gwosc.locate

`gwosc.locate` 在gwosc查找数据的网址

### gwosc.locate.get_event_urls

`gwosc.locate.get_event_urls` 提供指定引力波事件的数据的网址。

**说明**
```python

    get_event_urls(event, format='hdf5', sample_rate=4096, **match)
    
    Parameters
    ----------
    detector : `str`
        the prefix of the relevant GW detector
    dataset : `str`, `None`, optional
        the name of the dataset to query, e.g. ``'GW150914'``

    duration : `int`,`None`, optional
        the during of the dataset
    
    version : `int`, `None`, optional
        the data-release version for the selected datasets

    sample_rate : `int`, optional, default : ``4096``
        the sampling rate (Hz) of files you want to find

    format : `str`, optional, default: ``'hdf5'``
        the file format (extension) you want to find

    host : `str`, optional
        the URL of the remote LOSC server

    Returns
    -------
    urls : `list` of `str`
        the list of remote file URLs that contain data matching the
        relevant parameters
  
```    

#### 例子

例如获取GW150914的所有文件的网址

In [22]:
from gwosc.locate import get_event_urls
urls = get_event_urls('GW150914')
print(urls)

['https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5', 'https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5']


也可以指定探测器参数，时长等参数

In [50]:
urls = get_event_urls('GW150914', duration=32, detector='L1')
print(urls)

['https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5']
